In [1]:
import random
import json
import  pickle
import numpy as np

import nltk
from nltk.stem import WordNetLemmatizer

from sklearn.model_selection import train_test_split

from tensorflow.keras.models import  Sequential,load_model
from  tensorflow.keras.layers import Dense,Activation,Dropout
from tensorflow.keras.optimizers import Adam


In [2]:
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
import warnings
warnings.filterwarnings("ignore")

In [3]:
# nltk.download('punkt')
# nltk.download('stopwords')
# nltk.download('wordnet')
# nltk.download('omw-1.4')

In [4]:
lemmatizer  = WordNetLemmatizer()
intents = json.loads(open('data.json').read())

In [5]:
words = []
classes = []
documents = []
ignore_letters = ['?','!','.',',']

In [6]:

training = []


In [7]:
def clean_up_sentence(sentence):
    sentence_words =nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words if word not in ignore_letters]
    return sentence_words

In [8]:
for intent in intents['intents']:
    for pattern in intent['patterns']:
        word_list = clean_up_sentence(pattern)
        words.extend(word_list)
        documents.append((word_list,intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

#words = [lemmatizer.lemmatize(word) for word in words if word not in ignore_letters]
words = sorted(set(words))
classes = sorted(set(classes))

In [9]:
def bag_of_words(sentence_words):
    # sentence_words = clean_up_sentence(sentence)
    bag = [0]*len(words)
    for word in sentence_words:
        for i,w in enumerate(words):
            if w == word:
                bag[i]=1    
    return bag

In [10]:
for document in documents:
    # bag = []
    # word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]
    # for word in words : 
    #     bag.append(1) if word in word_patterns else bag.append(0)
    bag = bag_of_words(document[0])
    output_row = [0] * len(classes)
    output_row[classes.index(document[1])]=1
    training.append([bag,output_row])
    

In [11]:
random.shuffle(training)
training  = np.array(training)

In [12]:
X_train = list(training[:,0])
y_train = list(training[:,1])

model = Sequential()
model.add(Dense(128, input_shape=(len(X_train[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64,activation='relu'))
model.add(Dense(len(y_train[0]),activation='sigmoid'))


In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               9344      
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 64)                4160      
                                                                 
 dense_3 (Dense)             (None, 10)                650       
                                                                 
Total params: 22,410
Trainable params: 22,410
Non-traina

In [14]:
adam = Adam(lr=0.001)
model.compile(loss='categorical_crossentropy',optimizer=adam ,metrics=['accuracy'])

In [15]:
early_stop = EarlyStopping(monitor='loss', patience=2)
hist = model.fit(np.array(X_train),np.array(y_train),epochs=100,batch_size=5,verbose=2)

Epoch 1/100
9/9 - 1s - loss: 2.3593 - accuracy: 0.0476 - 658ms/epoch - 73ms/step
Epoch 2/100
9/9 - 0s - loss: 2.3236 - accuracy: 0.1190 - 12ms/epoch - 1ms/step
Epoch 3/100
9/9 - 0s - loss: 2.2812 - accuracy: 0.0952 - 12ms/epoch - 1ms/step
Epoch 4/100
9/9 - 0s - loss: 2.2006 - accuracy: 0.1429 - 17ms/epoch - 2ms/step
Epoch 5/100
9/9 - 0s - loss: 2.1768 - accuracy: 0.2381 - 14ms/epoch - 2ms/step
Epoch 6/100
9/9 - 0s - loss: 2.1726 - accuracy: 0.3333 - 14ms/epoch - 2ms/step
Epoch 7/100
9/9 - 0s - loss: 2.1530 - accuracy: 0.2619 - 14ms/epoch - 2ms/step
Epoch 8/100
9/9 - 0s - loss: 2.0929 - accuracy: 0.3095 - 12ms/epoch - 1ms/step
Epoch 9/100
9/9 - 0s - loss: 2.0787 - accuracy: 0.3571 - 17ms/epoch - 2ms/step
Epoch 10/100
9/9 - 0s - loss: 2.0492 - accuracy: 0.3333 - 16ms/epoch - 2ms/step
Epoch 11/100
9/9 - 0s - loss: 2.0197 - accuracy: 0.3095 - 13ms/epoch - 1ms/step
Epoch 12/100
9/9 - 0s - loss: 1.9108 - accuracy: 0.3810 - 16ms/epoch - 2ms/step
Epoch 13/100
9/9 - 0s - loss: 1.8713 - accuracy

In [16]:
model.save('model.h5',hist)

In [17]:
model = load_model('model.h5')

In [18]:
def predict_class(sentence):
    bow = bag_of_words(clean_up_sentence(sentence))
    res = model.predict(np.array([np.array(bow)]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r > ERROR_THRESHOLD]
    
    results.sort(key=lambda x: x[1],reverse=True)
    return_list  =[]
    
    for r in results:
        return_list.append({'intent':classes[r[0]], 'probabilty':str(r[1])})
    
    return return_list
        

In [19]:
def get_reposnse(intents_list,intent_json):
    tag = intents_list[0]['intent']
    list_of_intents = intent_json['intents']
    result =""
    for i in list_of_intents:
        if i['tag'] == tag:
            result =random.choice(i['responses'])
            break
    return result

In [20]:
print("strat!!!!")

while True:
    message = input("")
    ints = predict_class(message)
    res = get_reposnse(ints,intents)
    print(res)

strat!!!!
1/1 [==============================] - 0s 106ms/step
No, Sorry you can check our other products.
1/1 [==============================] - 0s 20ms/step
No, Sorry you can check our other products.
1/1 [==============================] - 0s 20ms/step
No, Sorry you can check our other products.
1/1 [==============================] - 0s 41ms/step
Bye! Come back again soon.
1/1 [==============================] - 0s 21ms/step
See you later, thanks for visiting
1/1 [==============================] - 0s 18ms/step
Bye! Come back again soon.
1/1 [==============================] - 0s 20ms/step
Good to see you, how can I help You
1/1 [==============================] - 0s 22ms/step
All types of fashionable traditional wears.
